In [1]:
from matplotlib import pyplot as plt
%matplotlib inline
# %matplotlib widget    # interactive 3d plots
from sklearn.svm import SVC
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import data_loader
import model_evaluation
import plotting_utils
import model_training

# Feature (MFCC) extraction (Dataset generation)

In [ ]:
# run this function if you need to generate a csv data set from scratch
#data_loader.generate_csv_dataset()

# Loading Dataset

In [2]:
# X, y, ID = data_loader.load_dataset()
X, y, ID = data_loader.load_dataset(X_path='data/feature_vector_based_mean_mfcc_and_std_mfcc_nfft_4096.csv')

# ID['0'].unique()    # actors
group_members= data_loader.get_k_fold_group_member()
X_k_folds, y_k_folds = data_loader.get_custom_k_folds(X, y, ID, group_members)
print(len(X_k_folds))

data loaded in 0.03503274917602539ms
          0         1         2         3         4         5         6  \
0 -1.151378  0.323772  0.530925  0.638346  0.895739  0.666624  0.524236   
1 -1.057279  0.372491  0.213538  1.175228  1.005635  0.579586  0.891717   
2 -1.056317  0.588818  0.473571  0.718794  0.632658  0.722399  0.725295   
3 -0.975371  0.463351  0.642362  0.625777  0.830235  1.027107  0.831593   
4 -1.362673  0.773267  0.683576  0.978332  0.916855  0.884359  0.543358   

          7         8         9  ...        70        71        72        73  \
0  1.034262 -0.875917 -0.411641  ... -1.146454 -0.818756 -0.898584 -0.978988   
1  0.649381 -1.014615  0.055115  ... -0.913220 -0.697130 -1.146434 -0.805239   
2  0.575853 -0.728913 -0.691686  ... -1.372340 -1.178163 -1.031987 -1.040342   
3  0.187928 -0.744877 -0.567497  ... -0.692343 -0.933693 -0.446771 -0.819266   
4  0.913303 -0.549810 -0.838744  ... -1.103766 -1.219467 -1.223234 -0.802060   

         74        75        76

# GridSearch SVM-RBF

In [45]:
# del UAR_surf, UAR_list

In [3]:
# C = [0.01, 0.05, 0.1, 1.0, 10, 100]
# gamma = [100, 10, 1, 0.1, 0.01, 0.001]

C = np.logspace(-1, 2, 12, endpoint=True)
gamma = np.logspace(-3.5, -1.5, 10, endpoint=True)

UAR_list = []
UAR_best = 0
UAR_surf = np.zeros((len(C),len(gamma)))

for i in range(len(C)):
    for j in range(len(gamma)):
        model = SVC(kernel = 'rbf', C=C[i], gamma=gamma[j])
        UAR,y_pr,y_tr,y_te = model_training.estimate_model(model, X_k_folds, y_k_folds)
        UAR_surf[i,j] = UAR
        if UAR_best<UAR:
            UAR_best,y_pred,y_true,y_test = UAR,y_pr,y_tr,y_te
            C_best = C[i]
            gamma_best = gamma[j]
            print(f'C = {C[i]:.6f}, gamma = {gamma[j]:.6f}')
        UAR_list.append(UAR) 

print(f'UAR_best = {UAR_best:.3f}, C_best = {C_best:.6f}, gamma_best = {gamma_best:.6f}')

100%|██████████| 5/5 [00:01<00:00,  4.11it/s]


UAR = 0.315
C = 0.100000, gamma = 0.000316


100%|██████████| 5/5 [00:01<00:00,  4.31it/s]


UAR = 0.320
C = 0.100000, gamma = 0.000527


100%|██████████| 5/5 [00:01<00:00,  4.49it/s]


...

UAR = 0.449


100%|██████████| 5/5 [00:01<00:00,  4.07it/s]


UAR = 0.433


100%|██████████| 5/5 [00:01<00:00,  4.04it/s]

UAR = 0.395
UAR_best = 0.480, C_best = 15.199111, gamma_best = 0.004084


## Result visualization

In [19]:
fig = plt.figure(dpi=150)
ax = plt.axes(projection='3d')

log_g, log_C = np.meshgrid(np.log10(gamma),np.log10(C))

# ax.plot_surface(np.log10(C), np.log10(gamma), UAR_surf, rstride=1, cstride=1,
#                 cmap='viridis', edgecolor='none')
ax.plot_surface(log_C, log_g, UAR_surf, rstride=1, cstride=1,
                cmap='viridis', edgecolor='none')                
# ax.scatter(log_C, log_g, UAR_surf, c=UAR_surf, cmap='viridis', linewidth=0.5)
# ax.bar3d(np.log10(C), np.log10(gamma), UAR_surf,dx=1, dy=1, dz=1)
ax.set_xlabel('$log_{10}(C)$')
ax.set_ylabel('$log_{10}(\gamma)$')
ax.set_title('UAR')
ax.view_init(25, 260)
plt.show()

AttributeError: Figure.set() got an unexpected keyword argument 'projection'

# Confusion matrix

In [ ]:
plotting_utils.plot_confusion_matrix(y_test, y_true, y_pred)